学習した教師モデルには温度 T で割る処理が入っていないので、teacher_model.layers.pop() を使用して出力層を外します。外した後に温度 T で割る層と出力層を追加します。今回はセグメンテーションのため、出力層に Sigmoid を追加します。

In [ ]:
# 教師モデル構築
from keras.models import load_model
teacher_model = load_model("teacher_model.h5", custom_objects={'mean_iou': mean_iou})
# 重み固定
for i in range(len(teacher_model.layers)):
    teacher_model.layers[i].trainable = False
teacher_model.compile(optimizer="adam", loss="binary_crossentropy")
# 教師モデルの出力層を削除
teacher_model.layers.pop()
input_layer = teacher_model.input
# 温度 T で割る処理
theacher_logits = teacher_model.layers[-1].output
theacher_logits_T = Lambda(lambda X: X/T)(theacher_logits)
teacher_probabilities_T = Activation('sigmoid')(theacher_logits_T)

生徒モデルの入力層は教師モデルの入力層を使用して共有させます。ここでは、温度T割る出力(Ys_soft)と割らない出力(Ys_hard)、二つの出力層を作成します。推論の際には温度Tで割らない出力(Ys_hard)を使用します。

In [ ]:
# 生徒モデル
s = Lambda(lambda x:x /255.0)(input_layer) # 教師モデルの入力層

~省略 (U-Net 構築) ~

output = Activation('sigmoid', name="output")(tc10) # 推論用(Ys-hard)
logits_T =Lambda(lambda X: X/T)(tc10)
probabilities_T  = Activation("sigmoid", name="probabilities_T")(logits_T) # soft target loss (Ys-soft)

student_model = Model(inputs=[input_layer], outputs=[output]) # 生徒モデル用に出力を
student_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])
student_model.summary()

先程作成した教師モデルと生徒モデルを同士に学習するモデルを作成します。「with tf.device('/cpu:0'):」を使用するとモデルを並列で処理するようになります。「knowledge_distillation_loss」は自作損失関数になります。今回は正解ラベルが「Ytrue」、「Yt_soft」の二つ。推論結果が「Ys_hard」、「Ys_soft」の二つ。変数が計4つあります。損失関数には引数が二つまでしか設定できないため、自作した損失関数をレイヤーに組込みます。そして、「教師モデル + 生徒モデル」の出力を損失値にします。「knowledge_distillation_loss」の実装は次に説明します。
「loss= lambda y_true, y_pred: y_pred」の部分は、コンパイルの損失の設定の部分に自作損失関数を入れず、モデルの出力が loss になるようにしています。

In [ ]:
# 生徒モデル
with tf.device('/cpu:0'):
    student_model = Model(inputs=input_layer, outputs=output)
    # 入力として学習データの正解ラベルを入れる
    input_true = Input(name='input_true', shape=[im_height, im_width, im_chan])

# 教師モデル + 生徒モデル
# 自作損失関数をレイヤーとして組込み
output_loss = Lambda(knowledge_distillation_loss, output_shape=(1,), name='kd_')(
    [output, input_true, teacher_probabilities_T, probabilities_T]
)

# input_layer：入力　input_true：学習データの正解ラベル
inputs = [input_layer, input_true] 
with tf.device('/cpu:0'):
    # 損失値を出力とする
    train_model = Model(inputs=inputs, outputs=output_loss)

# 出力が loss になるように設定
train_model.compile(optimizer='adam', loss= lambda y_true, y_pred: y_pred)

hard target loss + soft hard target loss の損失関数の数式②をコードにすると以下になります。

In [ ]:
from keras.losses import binary_crossentropy as logloss
lambda_ =  0.9
def knowledge_distillation_loss(input_distillation):
    y_pred, y_true, y_soft, y_pred_soft = input_distillation
    return (1 - lambda_) * logloss(y_true, y_pred) + lambda_*T*logloss(y_soft, y_pred_soft)